### Obtaining a data set

I've put a zipped csv file from Lending Tree containing mortage data on One Drive. You should all have access. You can find it here: https://glgit-my.sharepoint.com/:u:/g/personal/jobelenus_glgroup_com/EUoBwtK-k89KopT44j4DjqsB_N2IPj36kuZUmY7SpDgTwg?e=fgEzC0

I've added a column reference here: https://github.com/jobelenus/python-data-analysis-crash-course/blob/master/01-Pandas/reference.md.



### Analyzing the data

*Note: Skip Line 1! Line 2 is the header, so skip that too!*

1. Try and group the dataset by "grade" (A,B,C,D,F).
2. Then see if the highest interest rate in B is greater than the lowest in A, for each grade (e.g. compare a grade with the one above it)

1. Try and group the data by loan status and term, to determine whether more shorter mortgages are fully paid off than longer ones

1. Try and select all the A grade mortgages, and add a new column that calculates the total amount of dollars the buyer will owe (loan_amt * int_rate).
2. Then add another column that tells you how many years it would take to pay it off if they paid with their entire annual income each year.

Reading in csv - found issue with 3 columns - set the dtypes to object
Skipping 1 row not 2 since heading is created if not specified

In [16]:
import os
import pandas as pd
path = '/Users/dgriffis/code/glg/python-data-analysis-crash-course/data/LoanStats3d.csv.zip'
pd.show_versions()


INSTALLED VERSIONS
------------------
commit: None
python: 3.7.2.final.0
python-bits: 64
OS: Darwin
OS-release: 17.7.0
machine: x86_64
processor: i386
byteorder: little
LC_ALL: None
LANG: en_US.UTF-8
LOCALE: en_US.UTF-8

pandas: 0.24.1
pytest: None
pip: 19.0.2
setuptools: 40.8.0
Cython: None
numpy: 1.16.1
scipy: None
pyarrow: None
xarray: None
IPython: 7.2.0
sphinx: None
patsy: None
dateutil: 2.8.0
pytz: 2018.9
blosc: None
bottleneck: None
tables: None
numexpr: None
feather: None
matplotlib: 3.0.2
openpyxl: None
xlrd: None
xlwt: None
xlsxwriter: None
lxml.etree: None
bs4: None
html5lib: None
sqlalchemy: None
pymysql: None
psycopg2: None
jinja2: 2.10
s3fs: None
fastparquet: None
pandas_gbq: None
pandas_datareader: None
gcsfs: None


The next line reads in the zip file with the following:
loanStats_df = pd.read_csv(path, compression='infer', skiprows=1, dtype={"id": object, "desc": object, "verification_status_joint": object})

In [17]:
loanStats_df = pd.read_csv(path, compression='infer', skiprows=1, dtype={"id": object, "desc": object, "verification_status_joint": object})

In [18]:
grades = loanStats_df.groupby('grade');

In [19]:
len(grades)

7

In [21]:
#Try and group the dataset by "grade" (A,B,C,D,F).
grades.size()

grade
A     73336
B    117606
C    120567
D     62654
E     34948
F      9817
G      2167
dtype: int64